In [2]:
import tkinter as tk
from tkinter import filedialog
import threading
import time
import cv2
from ultralytics import YOLO

# --- Model YOLO ---
model = YOLO("E:/yolov8n.pt")

# --- GUI setup ---
window = tk.Tk()
window.title("Ngã tư 4 hướng")
window.geometry("420x440")
window.configure(bg="black")

canvas = tk.Canvas(window, width=400, height=400, bg="darkgray")
canvas.pack()

# Vẽ ngã tư
canvas.create_rectangle(150, 0, 250, 400, fill="gray")
canvas.create_rectangle(0, 150, 400, 250, fill="gray")

# --- Các đèn ---
west_red = canvas.create_oval(40, 190, 60, 210, fill="black")
west_yellow = canvas.create_oval(65, 190, 85, 210, fill="black")
west_green = canvas.create_oval(90, 190, 110, 210, fill="black")

east_red = canvas.create_oval(290, 190, 310, 210, fill="black")
east_yellow = canvas.create_oval(315, 190, 335, 210, fill="black")
east_green = canvas.create_oval(340, 190, 360, 210, fill="black")

north_red = canvas.create_oval(190, 40, 210, 60, fill="black")
north_yellow = canvas.create_oval(190, 65, 210, 85, fill="black")
north_green = canvas.create_oval(190, 90, 210, 110, fill="black")

south_red = canvas.create_oval(190, 290, 210, 310, fill="black")
south_yellow = canvas.create_oval(190, 315, 210, 335, fill="black")
south_green = canvas.create_oval(190, 340, 210, 360, fill="black")

# --- Label hiển thị ---
status_label = tk.Label(window, text="", font=("Arial", 12), bg="black", fg="white")
status_label.place(x=60, y=410)

# --- Hàm đổi màu đèn ---
def set_light(direction, color):
    lights = {
        "west": (west_red, west_yellow, west_green),
        "east": (east_red, east_yellow, east_green),
        "north": (north_red, north_yellow, north_green),
        "south": (south_red, south_yellow, south_green)
    }
    r, y, g = lights[direction]
    for light in (r, y, g):
        canvas.itemconfig(light, fill="black")
    if color == "red": canvas.itemconfig(r, fill="red")
    if color == "yellow": canvas.itemconfig(y, fill="yellow")
    if color == "green": canvas.itemconfig(g, fill="green")

# --- Hàm chọn ROI & đếm phương tiện ---
def detect_vehicles(direction, source_path):
    cap = cv2.VideoCapture(source_path)
    ret, frame = cap.read()
    if not ret:
        frame = cv2.imread(source_path)
        if frame is None:
            print(f"Không thể đọc file {source_path}")
            return {"motorbike": 0, "car": 0, "truck": 0, "bus": 0}

    drawing = False
    x1 = y1 = x2 = y2 = 0

    def draw_rectangle(event, x, y, flags, param):
        nonlocal drawing, x1, y1, x2, y2
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            x1, y1 = x, y
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            x2, y2 = x, y
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            x2, y2 = x, y

    temp_frame = frame.copy()
    cv2.namedWindow(f"Select ROI - {direction.upper()}")
    cv2.setMouseCallback(f"Select ROI - {direction.upper()}", draw_rectangle)

    while True:
        display = temp_frame.copy()
        if drawing or (x1 != x2 and y1 != y2):
            cv2.rectangle(display, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.imshow(f"Select ROI - {direction.upper()}", display)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == 27:  # ESC -> thoát toàn chương trình
            cv2.destroyAllWindows()
            window.destroy()
            exit()

    cv2.destroyAllWindows()

    if abs(x2 - x1) < 10 or abs(y2 - y1) < 10:
        print(f"ROI {direction.upper()} không hợp lệ.")
        return {"motorbike": 0, "car": 0, "truck": 0, "bus": 0}

    roi = frame[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]
    results = model(roi, verbose=False)
    counts = {"motorbike": 0, "car": 0, "truck": 0, "bus": 0}

    for box in results[0].boxes:
        cls_name = model.names[int(box.cls[0])]
        if cls_name in counts:
            counts[cls_name] += 1

    print(f"{direction.upper()}: {counts}")
    return counts

# --- Chu kỳ thông minh ---
def smart_cycle():
    time_weights = {"motorbike": 3, "car": 4, "truck": 5, "bus": 5}

    while True:
        # === CHỌN FILE CHO TUYẾN TÂY–ĐÔNG ===
        status_label.config(text="📁 Chọn video/ảnh cho tuyến Tây–Đông")
        window.update()
        west_east_path = filedialog.askopenfilename(
            title="Chọn video/ảnh Tây–Đông",
            filetypes=[("Media files", "*.mp4;*.jpg;*.png;*.jpeg")]
        )
        if not west_east_path:
            status_label.config(text="Không chọn file Tây–Đông. Dừng.")
            return

        # Quét Tây
        status_label.config(text="Quét ROI hướng Tây (nhấn Q khi xong)")
        window.update()
        count_west = detect_vehicles("west", west_east_path)

        # Quét Đông
        status_label.config(text="Quét ROI hướng Đông (nhấn Q khi xong)")
        window.update()
        count_east = detect_vehicles("east", west_east_path)

        total_time_west = sum(count_west[v] * time_weights[v] for v in time_weights)
        total_time_east = sum(count_east[v] * time_weights[v] for v in time_weights)
        avg_time_WE = max(3, int((total_time_west + total_time_east) / 2))

        # Hiển thị đèn xanh Tây–Đông
        status_label.config(text=f"Tây–Đông xanh {avg_time_WE}s (TB theo tổng giây)")
        set_light("west", "green")
        set_light("east", "green")
        for i in range(avg_time_WE, 0, -1):
            status_label.config(text=f"Tây–Đông còn {i}s (TB={avg_time_WE})")
            window.update()
            time.sleep(1)

        set_light("west", "yellow")
        set_light("east", "yellow")
        window.update()
        time.sleep(2)
        set_light("west", "red")
        set_light("east", "red")

        # === CHỌN FILE CHO TUYẾN BẮC–NAM ===
        status_label.config(text="Chọn video/ảnh cho tuyến Bắc–Nam")
        window.update()
        north_south_path = filedialog.askopenfilename(
            title="Chọn video/ảnh Bắc–Nam",
            filetypes=[("Media files", "*.mp4;*.jpg;*.png;*.jpeg")]
        )
        if not north_south_path:
            status_label.config(text="Không chọn file Bắc–Nam. Dừng.")
            return

        # Quét Bắc
        status_label.config(text="Quét ROI hướng Bắc (nhấn Q khi xong)")
        window.update()
        count_north = detect_vehicles("north", north_south_path)

        # Quét Nam
        status_label.config(text="Quét ROI hướng Nam (nhấn Q khi xong)")
        window.update()
        count_south = detect_vehicles("south", north_south_path)

        total_time_north = sum(count_north[v] * time_weights[v] for v in time_weights)
        total_time_south = sum(count_south[v] * time_weights[v] for v in time_weights)
        avg_time_NS = max(3, int((total_time_north + total_time_south) / 2))

        # Hiển thị đèn xanh Bắc–Nam
        status_label.config(text=f"Bắc–Nam xanh {avg_time_NS}s (TB theo tổng giây)")
        set_light("north", "green")
        set_light("south", "green")
        for i in range(avg_time_NS, 0, -1):
            status_label.config(text=f"Bắc–Nam còn {i}s (TB={avg_time_NS})")
            window.update()
            time.sleep(1)

        set_light("north", "yellow")
        set_light("south", "yellow")
        window.update()
        time.sleep(2)
        set_light("north", "red")
        set_light("south", "red")

        status_label.config(text="Chu kỳ mới bắt đầu...")
        window.update()

# --- Thread để GUI không bị đơ ---
def start_cycle():
    threading.Thread(target=smart_cycle).start()

# --- Nút khởi động ---
btn = tk.Button(window, text="Khởi động chu kỳ thông minh", command=start_cycle)
btn.place(x=110, y=380)

window.mainloop()

WEST: {'motorbike': 0, 'car': 5, 'truck': 1, 'bus': 0}
EAST: {'motorbike': 0, 'car': 1, 'truck': 1, 'bus': 0}
NORTH: {'motorbike': 0, 'car': 1, 'truck': 1, 'bus': 0}
SOUTH: {'motorbike': 0, 'car': 0, 'truck': 0, 'bus': 0}
WEST: {'motorbike': 0, 'car': 0, 'truck': 1, 'bus': 0}
EAST: {'motorbike': 0, 'car': 2, 'truck': 0, 'bus': 0}
